<a href="https://colab.research.google.com/github/shihongji/llm-search-agent/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment and libraies

In [3]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [4]:
import json
from typing import List, Tuple
from openai import OpenAI
import os
import requests
from google.colab import userdata

# OpenAI client
client = OpenAI(
  api_key=userdata.get('TOGETHER_API_KEY'),
  base_url='https://api.together.xyz/v1',
)

# You API web search
def get_ai_snippets_for_query(query):
    headers = {"X-API-Key": userdata.get('YOU_API_KEY')}
    params = {"query": query}
    results = requests.get(
        f"https://api.ydc-index.io/search",
        params=params,
        headers=headers,
    ).json()
    # return "\n".join(["\n".join(hit["snippets"]) for hit in results["hits"]])
    return results


#### Choose a model

In [5]:
model="mistralai/Mistral-7B-Instruct-v0.1"

## Research Agent

### Tools definition

In [84]:
topic_breakdown_tool = {
    "type": "function",
    "function": {
      "name": "break_down_topic",
      "description": "Take the broad research topic and break it down into 3 smaller, more focused subtopics or subqueries",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The broad research topic to break down"
          },
          "area": {
              "type": "string",
              "description": "The research area this topic belongs to"
          },
          "subtopics": {
              "type": "array",
              "description": "The 3 subtopics that the topic could be broke down into"
          }
        },
        "required": ["topic"]
      }
    }
}

query_expansion_tool = {
    "type": "function",
    "function": {
      "name": "expand_subtopic",
      "description": "Expand the subtopic to 3 total subqueries (e.g., related keywords, synonyms or phrases) to enhance the web search results",
      "parameters": {
        "type": "object",
        "properties": {
          "subtopic": {
              "type": "string",
              "description": "The sub-topic from a broader topic"
          },
          "subqueries": {
              "type": "string",
              "description": "The 3 subqueries related to the subtopic which are generated to enhance the future search results"
          }
        },
        "required": ["subtopic", "subqueries"]
      }
    }
}

combine_queries_tool = {
    "type": "function",
    "function": {
      "name": "combine_queries_tool",
      "description": "Combine expanded subqueries and keywords to generate the 3 most relevant queries to the main topic for enhanced search results",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The main research topic"
          },
          "subqueries": {
              "type": "array",
              "items": {
                  "type": "string"
              },
              "description": "The expanded subqueries and keywords related to the topic"
          },
            "search_queries": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "The 3 most relevant combined queries ready for search"
            }
        },
        "required": ["subqueries"]
      },
    }
}

write_essay_tool = {
    "type": "function",
    "function": {
      "name": "write_essay_on_research_topic",
      "description": "Write an essay around 800 words based on the summeries of search results on the research topic",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The main research topic"
          },
          "summeries": {
              "type": "array",
              "items": {
                  "type": "string"
              },
              "description": "summeries on some search results on the Internet"
          },
            "essay": {
                "type": "string",
                "description": "The essay written based on the summeries"
            }
        },
        "required": ["topic", "summeries"]
      },
    }
}
tools = [topic_breakdown_tool, query_expansion_tool, combine_queries_tool, write_essay_tool]

In [48]:
print(tools)

[{'type': 'function', 'function': {'name': 'break_down_topic', 'description': 'Take the broad research topic and break it down into 3 smaller, more focused subtopics or subqueries', 'parameters': {'type': 'object', 'properties': {'topic': {'type': 'string', 'description': 'The broad research topic to break down'}, 'area': {'type': 'string', 'description': 'The research area this topic belongs to'}, 'subtopics': {'type': 'array', 'description': 'The 3 subtopics that the topic could be broke down into'}}, 'required': ['topic']}}}, {'type': 'function', 'function': {'name': 'query_expansion_tool', 'description': 'Expand the subtopic to related keyword, synonyms and phrases (3 total) to enhance the web search results', 'parameters': {'type': 'object', 'properties': {'subtopic': {'type': 'string', 'description': 'The sub-topic from a broader topic'}, 'subqueries': {'type': 'array', 'description': 'The 3 subqueries related to the subtopic which are generated to enhance the future search resul

### Methods definition

In [89]:
def break_down_topic(research_topic: str) -> List[str]:
    """
    Breaks down the given research topic into subtopics using a tool (3 subtopics, in a list).
    @param research_topic (str): The research topic to be broken down.
    @return List[str]: A list of subtopics derived from the main topic.
    """
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": research_topic}
    ]
    breakdown_topics = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    parsed_data: dict = json.loads(breakdown_topics.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
    subtopics = parsed_data['subtopics']
    return subtopics

def expand_query(subtopics: List[str]) -> List[str]:
    """
    Given subtopics from breaking down the main topic, expand them to related keywords, phrases, 3 total
    @param subtopics List[str]: subtopics list from break_down_tool
    @return the expanded queries, each subtopic in a string, each string contains 3 related phrases, e.g.,:
    ['Climate change effects on military operations, Climate change and military planning, Climate change and military logistics',
    'climate change effects on logistics, climate change effects on supply chains, logistics and supply chain resilience to climate change',
    'climate change, conflict, security, environmental refugees, food insecurity, water scarcity, extreme weather events, natural disasters,
    displacement, migration, adaptation, mitigation']
    """
    expanded_queries = []
    for subtopic in subtopics:
        messages = [
            {"role": "system", "content": "You are a helpful research assistant."},
            {"role": "user", "content": f"Expand this {subtopic=} into subqueries (keywords, phrases... 3 total)"}
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        parsed_data: dict = json.loads(response.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
        expanded_query_info = parsed_data['subqueries']
        expanded_queries.append(expanded_query_info)
    return expanded_queries

def gen_queries(topic: str, subqueries: List[str]) -> List[str]:
    """
    To make efficient web search, combine subqueries from expand_query_tool into 3 most relevant queries.
    @param topic str: the main search topic
    @param subqueries List[str]: expanded queries from expand_query_tool
    @return queries_for_search List[str]: 3 most relevant queries, used for API web search, e.g.,
    ['Climate change effects on military operations and planning', 'Climate change and logistics resilience',
    'Climate change and conflict, security, and migration']
    """
    messages=[
        {"role": "system", "content": "You are a helpful research assistant. Help me combine the subqueries to 3 queries"},
        {"role": "user", "content": f"Combine {subqueries=} and main topic {topic=}, rephrase 3 most relevant search queries on the topic"}
    ]
    relevant_queries = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    parsed_data: dict = json.loads(relevant_queries.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
    queries_for_search = parsed_data['search_queries']
    return queries_for_search

def _collect_search_results(results) -> List[str]:
    sources = []
    stop_at: int = 2000
    flag = True
    for _ in results['hits']:
        if flag and len(_['snippets']) > 0:
            for s in _['snippets']:
                sources.append(s)
                stop_at -= len(s)
                # print(f"item in snippets lenth: {len(s)}, current words: {2000 - stop_at}")
                if stop_at < 0:
                    flag = False
                    break
    return sources

def _gen_summary(sub_query: str, sources: List[str]):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {sub_query} and search results {sources}, generate a 200 words summary on the topic"}
    ]
    print(messages)
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        tools=tools,
        tool_choice="auto",
    )
    print(result)
    # return relevant_queries
    # parsed_data: dict = json.loads(relevant_queries.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
    summary = json.dumps(result.choices[0].message.content)
    # queries_for_search = parsed_data['search_queries']
    # summary: dict = json.loads(result.choices[0].message.dict()['content'])
    return summary.strip()

def web_search(queries: List[str]) -> List[str]:
    summeries = []
    results = []
    for query in queries:
        results = get_ai_snippets_for_query(query)
        sources = _collect_search_results(results)
        summary = _gen_summary(query, sources)
        summeries.append(summary)
    return summeries

def write_research_essay(summeries: List[str], topic: str):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {topic} and search results summeries {summeries}, generate an 800 words essay on the topic"}
    ]
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=400,
        tools=tools,
        tool_choice="write_essay_on_research_topic",
    )
    print(result)
    return result

def critique_essay(essay: str, topic: str):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {topic} and essay: \n{essay}\n Please revise and improve the essay to make it more fluent and complete."}
    ]
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=400,
    )
    print(result)
    return result

## TEST

In [8]:
research_topic: str = "The impact of climate changes on World War"

### TEST Break down

In [ ]:
subtopics: List[str] = break_down_topic(research_topic)

### TEST Expand Subqueries

In [81]:
expanded_queries: List[str] = expand_query(subtopics)

[{'role': 'system', 'content': 'You are a helpful research assistant.'}, {'role': 'user', 'content': "Expand this subtopic='Climate change and its effects on military operations' into subqueries (keywords, phrases... 3 total)"}]
ChatCompletion(id='87665d599944677d-ATL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yz3bbikm29eezbzctl1mt3b4', function=Function(arguments='{"subtopic":"Climate change and its effects on military operations","subqueries":"Climate change effects on military operations, Climate change and military planning, Climate change and military logistics"}', name='expand_subtopic'), type='function')]))], created=1713461237, model='mistralai/Mistral-7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=72, prompt_tokens=956, total_tokens=1028), prompt=[])
[{'role

In [83]:
print(expanded_queries)

['Climate change effects on military operations, Climate change and military planning, Climate change and military logistics', 'climate change effects on logistics, climate change effects on supply chains, logistics and supply chain resilience to climate change', 'climate change, conflict, security, environmental refugees, food insecurity, water scarcity, extreme weather events, natural disasters, displacement, migration, adaptation, mitigation']


### TEST Generate optimized search queries

In [86]:
queries_for_search = gen_queries(research_topic, expanded_queries)

ChatCompletion(id='87666468cbd3b0db-ATL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uf9rmq9ku0h3by5p8dwex921', function=Function(arguments='{"topic":"The impact of climate changes on World War","subqueries":["Climate change effects on military operations, Climate change and military planning, Climate change and military logistics","climate change effects on logistics, climate change effects on supply chains, logistics and supply chain resilience to climate change","climate change, conflict, security, environmental refugees, food insecurity, water scarcity, extreme weather events, natural disasters, displacement, migration, adaptation, mitigation"],"search_queries":["Climate change effects on military operations and planning","Climate change and logistics resilience","Climate change and conflict, security, and migration"]}', name=

In [87]:
print(queries_for_search)

['Climate change effects on military operations and planning', 'Climate change and logistics resilience', 'Climate change and conflict, security, and migration']


### TEST Web search

In [ ]:
print(response)
# Need to parse the arguments string
parsed_data = json.loads(response.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
print(parsed_data['subtopics'])
type(parsed_data)

NameError: name 'response' is not defined

In [ ]:
# temp
print(queries)
summeries = []
for query in queries:
    results = get_ai_snippets_for_query(query)
    sources = collect_search_results(results)
    summary = gen_summary(query, sources)
    summeries.append(summary)



In [ ]:
test_break_down = break_down_topic("The impact of climate changes on World War")
print(test_break_down)

{
  "id": "875fd68a58918f14-IAD",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_8k01tvzvaaa37o0ujkasslqt",
            "function": {
              "arguments": "{\"topic\":\"The impact of climate changes on World War\",\"area\":\"Environmental Science\",\"subtopics\":[\"Climate change and its effects on military operations\",\"Climate change and its effects on logistics and supply chains\",\"Climate change and its effects on conflict and security\"]}",
              "name": "break_down_topic"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1713392801,
  "model": "mistralai/Mistral-7B-Instruct-v0.1",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 104,
    "prompt_tokens"

In [ ]:
test_expand_query = expand_query(test_break_down)
print(test_expand_query)
for m in test_expand_query:
    print(m)
# print(test_expand_query)

# 2D -> 1D
flat_queries = [item for child in test_expand_query for item in child]
# flat_queries = flat_queries[:6]
print(flat_queries)

[['Climate change and military operations', 'Military operations in the context of climate change', 'Climate change and its impact on military readiness'], ['Climate change and logistics', 'Climate change and supply chain management', 'Climate change and transportation'], ['Climate-induced migration and displacement', 'Climate-related natural disasters and conflict', 'Climate change and food security']]
['Climate change and military operations', 'Military operations in the context of climate change', 'Climate change and its impact on military readiness']
['Climate change and logistics', 'Climate change and supply chain management', 'Climate change and transportation']
['Climate-induced migration and displacement', 'Climate-related natural disasters and conflict', 'Climate change and food security']
['Climate change and military operations', 'Military operations in the context of climate change', 'Climate change and its impact on military readiness', 'Climate change and logistics', 'Cli

In [ ]:
len(flat_queries)

9

In [ ]:
print(json.dumps(response.choices[0].message.dict()['tool_calls'], indent=2))


[
  {
    "id": "call_yab3foz7qrl11tkqy0wjdypz",
    "function": {
      "arguments": "{\"topic\":\"The Reflection mechanism in Java language\",\"area\":\"Computer Science\",\"subtopics\":[\"Reflection API\",\"Reflection in Java\",\"Reflection vs. Introspection\"],\"subqueries\":[\"Reflection API in Java\",\"Reflection in Java programming\",\"Reflection vs. Introspection in Java\"]}",
      "name": "query_expansion"
    },
    "type": "function"
  }
]


### Analyze and Generate 3 most relavent queries

In [ ]:
# Legacy
combine_queries = {
    "type": "function",
    "function": {
      "name": "combine_queries",
      "description": "Combine expanded subqueries and keywords to generate the 3 most relevant queries to the topic for enhanced search results",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The broad research topic before breaking down"
          },
          "subqueries": {
              "type": "array",
              "items": {
                  "type": "string"
              },
              "description": "The expanded subqueries and keywords related to the topic"
          },
            "output": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "The 3 most relevant combined queries ready for search"
            }
        },
        "required": ["subqueries"]
      },
    }
}

In [ ]:
# test_break_down = break_down_topic("The impact of Stoicism on early Christianity")
test_topic = "The impact of climate changes on World War"
format_test_gen_queries = gen_queries_test(test_topic, flat_queries)
print(format_test_gen_queries)

[{'role': 'system', 'content': 'You are a helpful research assistant. Help me combine the subqueries to 3 queries'}, {'role': 'user', 'content': "given ['Climate change and military operations', 'Military operations in the context of climate change', 'Climate change and its impact on military readiness', 'Climate change and logistics', 'Climate change and supply chain management', 'Climate change and transportation', 'Climate-induced migration and displacement', 'Climate-related natural disasters and conflict', 'Climate change and food security'] and main topic The impact of climate changes on World War, rephrase 3 most relevant search queries on the topic"}]
ChatCompletion(id='875fdc02cd7f7ffa-IAD', choices=[Choice(finish_reason='eos', index=0, logprobs=None, message=ChatCompletionMessage(content=' 1. "Climate change and military operations"\n2. "Military operations in the context of climate change"\n3. "Climate change and its impact on military readiness"', role='assistant', function

TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(json.dumps(format_test_gen_queries.dict(), indent=2))

NameError: name 'format_test_gen_queries' is not defined

In [ ]:
# test_break_down = break_down_topic("The impact of Stoicism on early Christianity")
# test_topic = "The impact of Stoicism on early Christianit"
test_gen_queries = gen_queries(test_topic, flat_queries)

ChatCompletion(id='875fdfdc4de7802d-IAD', choices=[Choice(finish_reason='eos', index=0, logprobs=None, message=ChatCompletionMessage(content=' 1. "Climate change and military operations"\n2. "Climate change and logistics"\n3. "Climate change and transportation"', role='assistant', function_call=None, tool_calls=None))], created=1713393182, model='mistralai/Mistral-7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=33, prompt_tokens=1045, total_tokens=1078), prompt=[])


In [ ]:
a = test_gen_queries.choices[0].message.dict()['content'].split('\n')
b = [question.split('. ')[1].replace('"', '') for question in a]
# type(a)
print(a)
print(b)

[' 1. "Climate change and military operations"', '2. "Climate change and logistics"', '3. "Climate change and transportation"']
['Climate change and military operations', 'Climate change and logistics', 'Climate change and transportation']


### YOU Web Search

In [ ]:
# queries list
queries = b
type(queries)
test_you_search_result = get_ai_snippets_for_query(queries[0])

In [ ]:
type(test_you_search_result)

dict

In [ ]:
# temp
print(queries)
summeries = []
for query in queries:
    results = get_ai_snippets_for_query(query)
    sources = collect_search_results(results)
    summary = gen_summary(query, sources)
    summeries.append(summary)


['Climate change and military operations', 'Climate change and logistics', 'Climate change and transportation']
[{'role': 'system', 'content': 'You are a helpful research assistant.'}, {'role': 'user', 'content': "given the topic: Climate change and military operations and search results ['These threats come in forms both direct and indirect, including direct threats to military installations from sea level rise and extreme droughts, and indirect threats through the exacerbation of instability in critical regions. Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy.', 'Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy. Readiness: Readiness refers to the ability of a military to carry out operations in a timely manner. This involves having a stable and secure military infrastructure, including bases, supplies and logistics, in order to carry out missions. Climate change 

In [ ]:
print(len(summeries))
print(summeries[1])
len_word:int = 0
for _ in summeries:
    len_word += len(_)
print(len_word)

3
" The logistics industry is a significant contributor to global carbon emissions, with estimates suggesting that it accounts for around 24% of global CO2e emissions. As such, there is a growing focus on the need for climate-neutral logistics solutions that can help reduce the sector's carbon footprint. The European Environment Agency has set a target of reducing global logistics emissions by 40% by 2050, and policymakers have established a global framework for responding to dangerous climate change.\n\nOne of the key challenges facing the logistics industry is the need to adapt to changing weather patterns and other climate-related risks. For example, extreme weather events such as floods, wildfires, and droughts can disrupt supply chains and make it difficult for companies to move goods around the world. To address these challenges, many companies are turning to integrated logistics solutions that can help them stay resilient in the face of climate change.\n\n"
2512


In [ ]:
print(len(test_you_search_result))
print(queries[0])
print(test_you_search_result['hits'][0])
# print(test_you_search_result['hits'][0].items())
sources = []
stop_at: int = 2000
flag = True
for _ in test_you_search_result['hits']:
#     print(len(_))
    # print(_['description'])
    if flag and len(_['snippets']) > 0:
        for s in _['snippets']:
            sources.append(s)
            stop_at -= len(s)
            print(f"item in snippets lenth: {len(s)}, current words: {2000 - stop_at}")
            if stop_at < 0:
                flag = False
                break
    print(len(_['snippets']))
    print("====")

2
Climate change and military operations
{'description': "Air University is the U.S. Air and Space Force's center for professional military education (PME). Air University provides full-spectrum education, research and outreach at every level through professional military education, professional continuing education and academic degree granting.", 'snippets': [], 'title': 'The Implications of Climate Change for the Military and for ...', 'url': 'https://www.airuniversity.af.edu/Portals/10/ASPJ_French/journals_E/Volume-07_Issue-3/scottkhan_e.pdf'}
0
====
item in snippets lenth: 339, current words: 339
item in snippets lenth: 497, current words: 836
item in snippets lenth: 395, current words: 1231
item in snippets lenth: 566, current words: 1797
4
====
item in snippets lenth: 431, current words: 2228
4
====
4
====
4
====
1
====
4
====
0
====
1
====
0
====


In [ ]:
type(test_you_search_result)

dict

In [ ]:
for k, v in test_you_search_result['hits'][5].items():
    print(f"{k=}\t{v=}")

k='description'	v='Climate change is a national security threat to Europe. It is an accelerator of conflict and requires European forces to adapt accordingly. Concurrently, armed forces can help mitigate climate change by reducing their greenhouse-gas emissions. For states that plan to reduce carbon emissions, ...'
k='snippets'	v=['Climate change is a national security threat to Europe. It is an accelerator of conflict and requires European forces to adapt accordingly. Concurrently, armed forces can help mitigate climate change by reducing their greenhouse-gas emissions. For states that plan to reduce carbon emissions, decarbonisation of armed forces without disarming will be a challenge.']
k='title'	v='Green Defence: the defence and military implications of climate change for Europe'
k='url'	v='https://www.iiss.org/research-paper//2022/02/green-defence'


Summarize each snippet?

In [ ]:
current_sub_query = queries[0]
print(current_sub_query)
len(sources)
for i in sources:
    print(i)

Climate change and military operations
These threats come in forms both direct and indirect, including direct threats to military installations from sea level rise and extreme droughts, and indirect threats through the exacerbation of instability in critical regions. Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy.
Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy. Readiness: Readiness refers to the ability of a military to carry out operations in a timely manner. This involves having a stable and secure military infrastructure, including bases, supplies and logistics, in order to carry out missions. Climate change effects such as sea level rise have the ability to compromise coastal military installations that are critical for such operations.
Climate change effects such as sea level rise have the ability to compromise coastal military installations that are critic

In [ ]:
def gen_summary(sub_query: str, sources: List[str]):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {sub_query} and search results {sources}, generate a 200 words summary on the topic"}
    ]
    print(messages)
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        tools=tools,
        tool_choice="auto",
    )
    print(result)
    # return relevant_queries
    # parsed_data: dict = json.loads(relevant_queries.choices[0].message.dict()['tool_calls'][0]['function']['arguments'])
    summary = json.dumps(result.choices[0].message.content)
    # queries_for_search = parsed_data['search_queries']
    # summary: dict = json.loads(result.choices[0].message.dict()['content'])
    return summary.strip()

In [ ]:
test_summary = gen_summary(current_sub_query, sources)
print(test_summary)

[{'role': 'system', 'content': 'You are a helpful research assistant.'}, {'role': 'user', 'content': "given the topic: Climate change and military operations and search results ['These threats come in forms both direct and indirect, including direct threats to military installations from sea level rise and extreme droughts, and indirect threats through the exacerbation of instability in critical regions. Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy.', 'Climate change presents risks to three elements of military effectiveness: readiness, operations and strategy. Readiness: Readiness refers to the ability of a military to carry out operations in a timely manner. This involves having a stable and secure military infrastructure, including bases, supplies and logistics, in order to carry out missions. Climate change effects such as sea level rise have the ability to compromise coastal military installations that are critical f

In [ ]:
# Write essay
write_essay = {
    "type": "function",
    "function": {
      "name": "write_essay_on_research_topic",
      "description": "Write an essay around 800 words based on the summeries of search results on the research topic",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The main research topic"
          },
          "summeries": {
              "type": "array",
              "items": {
                  "type": "string"
              },
              "description": "summeries on some search results on the Internet"
          },
            "essay": {
                "type": "string",
                "description": "The essay written based on the summeries"
            }
        },
        "required": ["topic", "summeries"]
      },
    }
}
tools.append(write_essay)

In [ ]:
tools

In [ ]:
def write_research_essay(summeries: List[str], topic: str):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {topic} and search results summeries {summeries}, generate an 800 words essay on the topic"}
    ]
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=400,
        tools=tools,
        tool_choice="write_essay_on_research_topic",
    )
    print(result)
    return result

In [ ]:
test_essay = write_research_essay(summeries, test_topic)

ChatCompletion(id='87610d859f8682ae-IAD', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content=" The impact of climate change on World War is a complex issue that requires a multifaceted approach. Climate change poses significant risks to military effectiveness, including readiness, operations, and strategy. Readiness refers to the ability of a military to carry out operations in a timely manner, which can be compromised by climate change effects such as sea level rise and extreme weather events. Operations can also be impacted by climate change, whether they be war-fighting operations or humanitarian missions, with extreme droughts or flooding putting stresses on critical military infrastructure.\n\nThe logistics industry is a significant contributor to global carbon emissions, with estimates suggesting that it accounts for around 24% of global CO2e emissions. As such, there is a growing focus on the need for climate-neutral logistics s

TypeError: Object of type Choice is not JSON serializable

In [ ]:
temp_essay = json.dumps(test_essay.choices[0].message.content, indent=2)
print(temp_essay)


" The impact of climate change on World War is a complex issue that requires a multifaceted approach. Climate change poses significant risks to military effectiveness, including readiness, operations, and strategy. Readiness refers to the ability of a military to carry out operations in a timely manner, which can be compromised by climate change effects such as sea level rise and extreme weather events. Operations can also be impacted by climate change, whether they be war-fighting operations or humanitarian missions, with extreme droughts or flooding putting stresses on critical military infrastructure.\n\nThe logistics industry is a significant contributor to global carbon emissions, with estimates suggesting that it accounts for around 24% of global CO2e emissions. As such, there is a growing focus on the need for climate-neutral logistics solutions that can help reduce the sector's carbon footprint. The European Environment Agency has set a target of reducing global logistics emiss

In [ ]:
with open('temp_essay.txt', 'w') as f:
    f.write(temp_essay)

In [ ]:
def critique_essay(essay: str, topic: str):
    messages=[
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"given the topic: {topic} and essay: \n{essay}\n Please revise and improve the essay to make it more fluent and complete."}
    ]
    result = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=400,
    )
    print(result)
    return result

In [ ]:
test_critique = critique_essay(test_essay, test_topic)

ChatCompletion(id='8761245c09dc0664-IAD', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content=" The impact of climate change on World War is a complex issue that requires a multifaceted approach. Climate change poses significant risks to military effectiveness, including readiness, operations, and strategy. Readiness refers to the ability of a military to carry out operations in a timely manner, which can be compromised by climate change effects such as sea level rise and extreme weather events. Operations can also be impacted by climate change, whether they be war-fighting operations or humanitarian missions, with extreme droughts or flooding putting stresses on critical military infrastructure.\n\nThe logistics industry is a significant contributor to global carbon emissions, with estimates suggesting that it accounts for around 24% of global CO2e emissions. As such, there is a growing focus on the need for climate-neutral logistics s

In [ ]:
temp_critique = json.dumps(test_critique.choices[0].message.content, indent=2)
print(temp_critique)

" The impact of climate change on World War is a complex issue that requires a multifaceted approach. Climate change poses significant risks to military effectiveness, including readiness, operations, and strategy. Readiness refers to the ability of a military to carry out operations in a timely manner, which can be compromised by climate change effects such as sea level rise and extreme weather events. Operations can also be impacted by climate change, whether they be war-fighting operations or humanitarian missions, with extreme droughts or flooding putting stresses on critical military infrastructure.\n\nThe logistics industry is a significant contributor to global carbon emissions, with estimates suggesting that it accounts for around 24% of global CO2e emissions. As such, there is a growing focus on the need for climate-neutral logistics solutions that can help reduce the sector's carbon footprint. The European Environment Agency has set a target of reducing global logistics emiss

In [ ]:
with open('temp_critique.txt', 'w') as f:
    f.write(temp_critique)

In [ ]:
var0 = json.loads(test_summary.choices[0].message.dict()['content'].strip())
# var1 = json.dumps(test_summary.choices[0].message.content)
print(var1)
print(type(var1))
print(len(var1))
print(var0)

AttributeError: 'str' object has no attribute 'choices'

The length of search results is too big.
1. Create a function that iterates over the 10 search results and combine "snippets" field into a summary array -> summaries`[summary0, summary1....]`
2. Create a function that use the model to generate an essay.

---

How about 1. summary on the 1st search result, then use critique agent to assess whether or not continue to complement the essay or trigger another search.